In [1]:
import sys
sys.path.append('../utils')
sys.path.append('../core_update')
sys.path.append('../')
import numpy as np
import scipy
import tqdm
import utils
import geometry
from core_update import *

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import glob

Densities in g/cm^3

Paper 1:

HU < -365         rho = 1.0491 + 1.0526e-3 HU  
-365 < HU < 255         rho = 1.0053 + 8.921e-3 HU 
HU > 255         rho = 1.0610 + 6.813e-4 HU 


Paper 2:

HU < 148         rho = 1.0168 + 9.836e-4 HU 
HU > 148         rho = 1.1786 + 2.216e-4 HU 

Paper 3:

HU < 104         rho = 1.0050 + 0.0010 HU 
HU > 104         rho = 1.0152 + 0.0006 HU

In [ ]:
def transfer_function(image, method='first'):
    if method = 'first':
        density = np.zeros_like(image)
        density = np.where(image < -365, 1.0491 + 1.0526e-3 * image, density)
        density = np.where(image >= -365, 1.0053 + 8.921e-3 * image, density)
        density = np.where(image >= 255, 1.0610 + 6.813e-4 * image, density)
    elif method = 'second':
        density = np.zeros_like(image)
        density = np.where(image < 148, 1.0168 + 9.836e-4 * image, density)
        density = np.where(image >= 148, 1.1786 + 2.216e-4 * image, density)
    elif method = 'third':
        density = np.zeros_like(image)
        density = np.where(image < 104, 1.0050 + 0.0010 * image, density)
        density = np.where(image >= 104, 1.0152 + 0.0006 * image, density)
    else:
        print("transfer function method not recognized, supply as one of [first, second, third]")
        return 0
    return density

In [ ]:
def create_from_image(self, image, input_voxel_size, target_voxel_size=None, transfer_fn=None):
    if target_voxel_size is None:
        target_voxel_size = self.voxel_dims
    if transfer_fn is None:
        transfer_fn = lambda x: (x + np.amin(x)) / (np.amax(x) - np.amin(x))
        
    if type(image) == np.ndarray:
        data = image
    
    # data = transfer_fn(data)

    x = np.arange(0, data.shape[0])
    y = np.arange(0, data.shape[1])
    z = np.arange(0, data.shape[2])

    assert len(input_voxel_size) == 3, 'input voxel size must be a tuple of length 3'
    assert len(target_voxel_size) == 3, 'target voxel size must be a tuple of length 3'

    transformed_x = x * input_voxel_size[0] / target_voxel_size[0]
    transformed_y = y * input_voxel_size[1] / target_voxel_size[1]
    transformed_z = z * input_voxel_size[2] / target_voxel_size[2]

    from scipy.interpolate import RegularGridInterpolator
    interp = RegularGridInterpolator((transformed_x, transformed_y, transformed_z), data)
    # interp = NearestNDInterpolator((transformed_x, transformed_y, transformed_z), data)

    points = np.stack(np.meshgrid(np.arange(0, transformed_x[-1]), np.arange(0, transformed_y[-1]), np.arange(0, transformed_z[-1]), indexing='ij'), axis=-1)
    
    if points.shape[0] * points.shape[1] * points.shape[2] > 5e8:
        print('desired phantom array size is very large (>500,000,000 voxels), consider supplying a larger target_voxel_size or cropping the input image')
    if points.shape[0] * points.shape[1] * points.shape[2] > 2e9:
        print('desired phantom array size is too large (>2e9 voxels), consider supplying a larger target_voxel_size or cropping the input image')
        return 0
    
    new_phantom = interp(points)
    self.complete = np.stack((new_phantom * self.baseline[0], new_phantom * self.baseline[1]), axis = 0)    